# Logistic regression using `statsmodels`

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline

Read in the [British Crime Survey 2007-2008](https://beta.ukdataservice.ac.uk/datacatalogue/studies/study?id=6561) dataset.

In [2]:
bcs = pd.read_csv("datasets/bcs.csv")

In [3]:
bcs.head()

,case_id,sex,age,marital_status,ethnic_origin,education,paid_work_last_wk,years_in_area,same_address_last_year,tenure,...,worry_rape,worry_assault,worry_insult,worry_racism,crime_change_two_yrs,personal_crime,property_crime,antisocial_behaviour,nuisance_neighbours,victim_last_yr
0,61302140,F,36.0,Married,White,None,True,"[10, 20)",NaN,Buying with mortgage/loan,...,Not very worried,Not very worried,Fairly worried,Not very worried,A little more crime,NaN,NaN,1.117700,2.212788,False
1,61384060,M,44.0,Separated,White,Apprenticeship or A/AS Level,True,"[0, 1)",False,Rent,...,Not at all worried,Not very worried,Not very worried,Not very worried,NaN,-0.389274,2.139811,1.791787,-1.024336,False
2,63684260,M,43.0,Married,White,O Level / GCSE,True,"[2, 3)",NaN,Buying with mortgage/loan,...,Not at all worried,Not very worried,Not very worried,Not very worried,NaN,-0.320777,0.506092,NaN,NaN,True
3,63790220,F,27.0,Single,Black or Black British,Apprenticeship or A/AS Level,True,"[0, 1)",False,Rent,...,Fairly worried,Fairly worried,Not very worried,Not very worried,NaN,0.464605,1.602307,0.462970,0.822929,True
4,63843180,M,38.0,Married,White,Degree or Diploma,True,"[10, 20)",NaN,Buying with mortgage/loan,...,Not at all worried,Not very worried,Not very worried,Not very worried,A lot more crime,-0.117556,-0.657555,0.137419,-0.780199,True


Define predictors and response.

In [4]:
predictors = ["sex", "age", "safety_walk_night"]
response = "victim_last_yr"
all_vars = predictors + [response]

Remove missing values.

In [5]:
bcs.dropna(subset=all_vars, inplace=True)

## EDA

In [6]:
sns.boxplot(x="victim_last_yr", y="age", data=bcs)

In [7]:
bcs.groupby("safety_walk_night")["victim_last_yr"].mean().plot.bar(color="darkblue")

## Logistic regression

In [8]:
bcs["victim_last_yr"] = bcs["victim_last_yr"].astype(int)  # Recode as 0/1

In [9]:
model = smf.glm(
    "victim_last_yr ~ C(sex, Treatment(reference='M')) + age + C(safety_walk_night, Treatment(reference='Very safe'))",
    data=bcs,
    family=sm.families.Binomial()
).fit()

In [10]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:         victim_last_yr   No. Observations:                11610
Model:                            GLM   Df Residuals:                    11604
Model Family:                Binomial   Df Model:                            5
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -5550.0
Date:                Wed, 30 Jan 2019   Deviance:                       11100.
Time:                        08:49:25   Pearson chi2:                 1.15e+04
No. Iterations:                     5   Covariance Type:             nonrobust
==========================================================================================================================================
                                                                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                 -0.1315      0.079     -1.666      0.096      -0.286       0.023
C(sex, Treatment(reference='M'))[T.F]                                     -0.1883      0.051     -3.704      0.000      -0.288      -0.089
C(safety_walk_night, Treatment(reference='Very safe'))[T.A bit unsafe]     0.6191      0.072      8.657      0.000       0.479       0.759
C(safety_walk_night, Treatment(reference='Very safe'))[T.Fairly safe]      0.2184      0.063      3.456      0.001       0.095       0.342
C(safety_walk_night, Treatment(reference='Very safe'))[T.Very unsafe]      0.7749      0.089      8.680      0.000       0.600       0.950
age                                                                       -0.0308      0.001    -22.432      0.000      -0.033      -0.028
==========================================================================================================================================
"""

In [11]:
np.exp(model.params)

Intercept                                                                 0.876819
C(sex, Treatment(reference='M'))[T.F]                                     0.828330
C(safety_walk_night, Treatment(reference='Very safe'))[T.A bit unsafe]    1.857319
C(safety_walk_night, Treatment(reference='Very safe'))[T.Fairly safe]     1.244052
C(safety_walk_night, Treatment(reference='Very safe'))[T.Very unsafe]     2.170337
age                                                                       0.969700
dtype: float64

In [12]:
np.exp(model.conf_int())

,0,1
Intercept,0.751161,1.023499
"C(sex, Treatment(reference='M'))[T.F]",0.749761,0.915132
"C(safety_walk_night, Treatment(reference='Very safe'))[T.A bit unsafe]",1.614393,2.136799
"C(safety_walk_night, Treatment(reference='Very safe'))[T.Fairly safe]",1.099151,1.408054
"C(safety_walk_night, Treatment(reference='Very safe'))[T.Very unsafe]",1.821952,2.585338
age,0.967096,0.972310


In [13]:
model.pvalues

Intercept                                                                  9.578223e-02
C(sex, Treatment(reference='M'))[T.F]                                      2.120706e-04
C(safety_walk_night, Treatment(reference='Very safe'))[T.A bit unsafe]     4.847630e-18
C(safety_walk_night, Treatment(reference='Very safe'))[T.Fairly safe]      5.477788e-04
C(safety_walk_night, Treatment(reference='Very safe'))[T.Very unsafe]      3.964205e-18
age                                                                       1.918834e-111
dtype: float64